In [2]:
import ast
import pandas as pd
from typing import Union
from fastapi import FastAPI
from datetime import datetime
import json

### Importar el archivo ##############################################
df = pd.read_csv("movies_dataset.csv")


C:\Users\Martin\AppData\Local\Temp\ipykernel_20688\2777016581.py:9: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("movies_dataset.csv")


In [3]:
####################### DESANIDADO #####################################

### Desanidado "belong_to_collection"####################################

# Convertir la columna 'belongs_to_collection' en una lista de diccionarios
df['belongs_to_collection'] = df['belongs_to_collection'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None)

# Crear nuevas columnas
df['collection_id'] = df['belongs_to_collection'].apply(
    lambda x: x['id'] if isinstance(x, dict) else None)
df['collection_name'] = df['belongs_to_collection'].apply(
    lambda x: x['name'] if isinstance(x, dict) else None)
df['poster_path'] = df['belongs_to_collection'].apply(
    lambda x: x['poster_path'] if isinstance(x, dict) else None)
df['backdrop_path'] = df['belongs_to_collection'].apply(
    lambda x: x['backdrop_path'] if isinstance(x, dict) else None)


In [4]:

### Desanidado "production_countries" ###################################

# Convertir la columna 'production_companies' en una lista de diccionarios
df['production_companies'] = df['production_companies'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None)

# Crear nuevas columnas desanidadas para 'production_companies'
df['company_names'] = df['production_companies'].apply(
    lambda x: [company['name'] for company in x] if isinstance(x, list) else [])
df['company_ids'] = df['production_companies'].apply(
    lambda x: [company['id'] for company in x] if isinstance(x, list) else [])


In [5]:
### Desanidado "genres" #################################

df['genre_names'] = df['genres'].apply(
    lambda x: [genre['name'] for genre in eval(x)])


In [6]:
### Desanidado "production_countries" ####################################

# Convertir la columna 'production_countries' en una lista de diccionarios
df['production_countries'] = df['production_countries'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Desanidar la columna 'production_countries'
df['production_countries'] = df['production_countries'].apply(
    lambda x: [country['name'] for country in x] if isinstance(x, list) else [])


In [8]:
### Transformacion columnas revenue y budget ##############################

df['budget'] = pd.to_numeric(
    df['budget'], errors='coerce').fillna(0).astype(int)
df['revenue'] = pd.to_numeric(
    df['revenue'], errors='coerce').fillna(0).astype(int)


# Convercion de las columnas "budget"  y "revenue" a un tipo de datos numérico

df['budget'] = df['budget'].astype(int)
df['revenue'] = df['revenue'].astype(int)





In [10]:
### Transoformacion de la columna release_date"##############################################

# Convertir la columna "release_date" al formato de fecha
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Darle formato AAAA-mm-dd a 'release_date'
df['release_date'] = df['release_date'].dt.strftime('%Y-%m-%d')  # 2

# Crear la columna "release_year" para extraer el año de la fecha de estreno
df['release_year'] = df['release_date'].str[:4]  # 2

# Eliminar filas con valores nulos en la columna "release_date"
df.dropna(subset=['release_date'], inplace=True)




In [9]:
### Columna "return"##########################################

# Calcular el retorno de inversión solo cuando revenue y budget son diferentes de 0

df['return'] = df.apply(lambda row: row['revenue'] /
                        row['budget'] if row['budget'] != 0 else 0, axis=1)


In [10]:
### Eliminación de columnas no utilizadas y/o desanidadas ##############################
df = df.drop(columns=["belongs_to_collection", "genres","video", "imdb_id", "adult",
             "original_title", "vote_count", "poster_path", "homepage", 'id', 'overview', 'production_companies',
                      'spoken_languages', 'tagline', 'collection_id', 'poster_path', 'backdrop_path'])


In [ ]:
df.to_csv('dataset_Transformado.csv', sep=';', index=False)

In [11]:
df.head(20)

,budget,original_language,popularity,production_countries,release_date,revenue,runtime,status,title,vote_average,collection_name,company_names,company_ids,genre_names,return
0,30000000,en,21.946943,[United States of America],1995-10-30,373554033,81.0,Released,Toy Story,7.7,Toy Story Collection,[Pixar Animation Studios],[3],"[Animation, Comedy, Family]",12.451801
1,65000000,en,17.015539,[United States of America],1995-12-15,262797249,104.0,Released,Jumanji,6.9,None,"[TriStar Pictures, Teitler Film, Interscope Co...","[559, 2550, 10201]","[Adventure, Fantasy, Family]",4.043035
2,0,en,11.7129,[United States of America],1995-12-22,0,101.0,Released,Grumpier Old Men,6.5,Grumpy Old Men Collection,"[Warner Bros., Lancaster Gate]","[6194, 19464]","[Romance, Comedy]",0.000000
3,16000000,en,3.859495,[United States of America],1995-12-22,81452156,127.0,Released,Waiting to Exhale,6.1,None,[Twentieth Century Fox Film Corporation],[306],"[Comedy, Drama, Romance]",5.090760
4,0,en,8.387519,[United States of America],1995-02-10,76578911,106.0,Released,Father of the Bride Part II,5.7,Father of the Bride Collection,"[Sandollar Productions, Touchstone Pictures]","[5842, 9195]",[Comedy],0.000000
5,60000000,en,17.924927,[United States of America],1995-12-15,187436818,170.0,Released,Heat,7.7,None,"[Regency Enterprises, Forward Pass, Warner Bros.]","[508, 675, 6194]","[Action, Crime, Drama, Thriller]",3.123947
6,58000000,en,6.677277,"[Germany, United States of America]",1995-12-15,0,127.0,Released,Sabrina,6.2,None,"[Paramount Pictures, Scott Rudin Productions, ...","[4, 258, 932, 5842, 14941, 55873, 58079]","[Comedy, Romance]",0.000000
7,0,en,2.561161,[United States of America],1995-12-22,0,97.0,Released,Tom and Huck,5.4,None,[Walt Disney Pictures],[2],"[Action, Adventure, Drama, Family]",0.000000
8,35000000,en,5.23158,[United States of America],1995-12-22,64350171,106.0,Released,Sudden Death,5.5,None,"[Universal Pictures, Imperial Entertainment, S...","[33, 21437, 23770]","[Action, Adventure, Thriller]",1.838576
9,58000000,en,14.686036,"[United Kingdom, United States of America]",1995-11-16,352194034,130.0,Released,GoldenEye,6.6,James Bond Collection,"[United Artists, Eon Productions]","[60, 7576]","[Adventure, Action, Thriller]",6.072311
